# Data Mining Project : Analysis of intentional homicides by country and sex of victims 

In [1]:
# Data analysis imports
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np
from ydata_profiling import ProfileReport

# ML imports
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import BayesianGaussianMixture
from sklearn.cluster import DBSCAN
from scipy.stats import norm
import matplotlib.pyplot as plt
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer

pd.set_option('display.max_columns', None)

/home/ades/miniconda3/envs/pytorchgeo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prétraitement des données

### Transformation fichier en csv avec renommage

In [2]:
# Read the Excel file with the first two rows as header
read_file = pd.read_excel("./datasets/Intentional homicide victims by sex, counts and ra.xls", sheet_name=2, header=[0, 1])

# Write the dataframe object into a CSV file
read_file.to_csv("./datasets/raw_data.csv", index=None, header=True)

# Read the CSV file back into a dataframe
df = pd.read_csv('./datasets/raw_data.csv', header=[0, 1])

# Rename Highest level columns
df.columns = pd.MultiIndex.from_tuples(
    [(('Count' if 'Number of victims of intentional homicides by sex' in col else 'Rate' if 'Rates of intentional homicides by sex' in col else col[0]), col[1]) for col in df.columns]
)
df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if col[0] != 'Unnamed: 0_level_0' else 'Info', col[1]) for col in df.columns]
)
df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if col[0] != 'Unnamed: 1_level_0' else 'Info', col[1]) for col in df.columns]
)
df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if col[0] != 'Unnamed: 2_level_0' else 'Info', col[1]) for col in df.columns]
)
df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if col[0] != 'Unnamed: 3_level_0' else 'Info', col[1]) for col in df.columns]
)
df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if col[0] != 'Unnamed: 4_level_0' else 'Info', col[1]) for col in df.columns]
)

df.head(10)


Info                                                     Count            \
   Region           Subregion   Country     Sex        Source  2000 2001 2002   
0  Africa     Northern Africa   Algeria  Female           CTS   NaN  NaN  NaN   
1  Africa     Northern Africa   Algeria    Male           CTS   NaN  NaN  NaN   
2  Africa     Northern Africa     Egypt  Female  CTS/External   NaN  NaN  NaN   
3  Africa     Northern Africa     Egypt    Male           CTS   NaN  NaN  NaN   
4  Africa     Northern Africa   Morocco  Female     UNSDC/CTS   NaN  NaN  NaN   
5  Africa     Northern Africa   Morocco    Male     UNSDC/CTS   NaN  NaN  NaN   
6  Africa     Northern Africa   Tunisia  Female      External   NaN  NaN  NaN   
7  Africa     Northern Africa   Tunisia    Male      External   NaN  NaN  NaN   
8  Africa  Sub-Saharan Africa  Botswana  Female            NP   NaN  NaN  NaN   
9  Africa  Sub-Saharan Africa  Botswana    Male            NP   NaN  NaN  NaN   

                                                                            \
  2003   2004   2005   2006   2007        2008        2009    2010    2011   
0  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   
1  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   
2  NaN   52.0   63.0  144.0  163.0  110.418685  193.000000   197.0   331.0   
3  NaN  270.0  459.0  405.0  517.0  856.581315  719.000000  1642.0  2372.0   
4  NaN    NaN    NaN    NaN    NaN         NaN   52.897959     NaN     NaN   
5  NaN    NaN    NaN    NaN    NaN         NaN  379.102041     NaN     NaN   
6  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   
7  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   
8  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   
9  NaN    NaN    NaN    NaN    NaN         NaN         NaN     NaN     NaN   

                                                                             \
          2012   2013        2014   2015   2016   2017   2018   2019   2020   
0    52.000000   39.0   85.481481   71.0   75.0   56.0   86.0   86.0  122.0   
1   195.000000  170.0  491.518519  475.0  467.0  468.0  482.0  433.0  532.0   
2   261.865211    NaN         NaN    NaN    NaN    NaN    NaN    NaN  165.0   
3  1945.134789    NaN         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4    56.000000   66.0   59.000000   97.0  105.0  113.0   70.0  117.0   84.0   
5   327.000000  363.0  296.000000  334.0  489.0  648.0  440.0  510.0  374.0   
6          NaN    NaN         NaN    NaN    NaN    NaN    NaN  103.0  115.0   
7          NaN    NaN         NaN    NaN    NaN    NaN    NaN  468.0  447.0   
8          NaN    NaN         NaN    NaN    NaN    NaN    NaN    NaN  114.0   
9          NaN    NaN         NaN    NaN    NaN    NaN    NaN    NaN    5.0   

         Rate                                                         \
    2021 2000 2001 2002 2003      2004      2005      2006      2007   
0  160.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
1  535.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
2  296.0  NaN  NaN  NaN  NaN  0.135997  0.161524  0.362020  0.401793   
3    NaN  NaN  NaN  NaN  NaN  0.687261  1.145443  0.991362  1.241278   
4   99.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
5  617.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
6    NaN  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
7    NaN  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
8  100.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   
9    5.0  NaN  NaN  NaN  NaN       NaN       NaN       NaN       NaN   

                                                                         \
       2008      2009      2010      2011      2012      2013      2014   
0       NaN       NaN       NaN       NaN  0.284704  0.209345  0.449798   
1       NaN       NaN       NaN       NaN  1.026532  0.8

### Filtrer les sources et arrondir le nombre de victimes

In [3]:
# Filter out sources from the dataframe
df = df.iloc[:327]
# How to select Country
print(df[('Info', 'Country')])
# How to filter rows based on Country 
df_algeria = df[df[('Info', 'Country')] == 'Algeria']
print(df_algeria)

# Round Counts to nearest integer
rounded_counts = df['Count'].round(0)
df['Count'] = rounded_counts

0             Algeria
1             Algeria
2               Egypt
3               Egypt
4             Morocco
            ...      
322    American Samoa
323             Samoa
324             Samoa
325             Tonga
326             Tonga
Name: (Info, Country), Length: 327, dtype: object
     Info                                          Count                      \
   Region        Subregion  Country     Sex Source  2000 2001 2002 2003 2004   
0  Africa  Northern Africa  Algeria  Female    CTS   NaN  NaN  NaN  NaN  NaN   
1  Africa  Northern Africa  Algeria    Male    CTS   NaN  NaN  NaN  NaN  NaN   

                                                                              \
  2005 2006 2007 2008 2009 2010 2011   2012   2013        2014   2015   2016   
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   52.0   39.0   85.481481   71.0   75.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  195.0  170.0  491.518519  475.0  467.0   

                                     Rate                         

### Ajouter des colonnes au DataFrame (Somme & moyenne du nb de victimes, Somme & moyenne du taux meurtre)

In [4]:
# Add a total victim count column to the dataframe
df[('Count', 'Total')] = df[['Count']].sum(axis=1)
df.head(10)

# Add the mean rate column to the dataframe
df[('Count', 'Mean')] = df[['Count']].mean(axis=1)
df.head(10)

# Add a total victim count column to the dataframe
df[('Rate', 'Total')] = df[['Rate']].sum(axis=1)
df.head(10)

# Add the mean rate column to the dataframe
df[('Rate', 'Mean')] = df[['Rate']].mean(axis=1)
df.head(10)

Info                                                     Count            \
   Region           Subregion   Country     Sex        Source  2000 2001 2002   
0  Africa     Northern Africa   Algeria  Female           CTS   NaN  NaN  NaN   
1  Africa     Northern Africa   Algeria    Male           CTS   NaN  NaN  NaN   
2  Africa     Northern Africa     Egypt  Female  CTS/External   NaN  NaN  NaN   
3  Africa     Northern Africa     Egypt    Male           CTS   NaN  NaN  NaN   
4  Africa     Northern Africa   Morocco  Female     UNSDC/CTS   NaN  NaN  NaN   
5  Africa     Northern Africa   Morocco    Male     UNSDC/CTS   NaN  NaN  NaN   
6  Africa     Northern Africa   Tunisia  Female      External   NaN  NaN  NaN   
7  Africa     Northern Africa   Tunisia    Male      External   NaN  NaN  NaN   
8  Africa  Sub-Saharan Africa  Botswana  Female            NP   NaN  NaN  NaN   
9  Africa  Sub-Saharan Africa  Botswana    Male            NP   NaN  NaN  NaN   

                                                                          \
  2003   2004   2005   2006   2007   2008   2009    2010    2011    2012   
0  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN    52.0   
1  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN   195.0   
2  NaN   52.0   63.0  144.0  163.0  110.0  193.0   197.0   331.0   262.0   
3  NaN  270.0  459.0  405.0  517.0  857.0  719.0  1642.0  2372.0  1945.0   
4  NaN    NaN    NaN    NaN    NaN    NaN   53.0     NaN     NaN    56.0   
5  NaN    NaN    NaN    NaN    NaN    NaN  379.0     NaN     NaN   327.0   
6  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN   
7  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN   
8  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN   
9  NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN   

                                                                 Rate       \
    2013   2014   2015   2016   2017   2018   2019   2020   2021 2000 2001   
0   39.0   85.0   71.0   75.0   56.0   86.0   86.0  122.0  160.0  NaN  NaN   
1  170.0  492.0  475.0  467.0  468.0  482.0  433.0  532.0  535.0  NaN  NaN   
2    NaN    NaN    NaN    NaN    NaN    NaN    NaN  165.0  296.0  NaN  NaN   
3    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  NaN  NaN   
4   66.0   59.0   97.0  105.0  113.0   70.0  117.0   84.0   99.0  NaN  NaN   
5  363.0  296.0  334.0  489.0  648.0  440.0  510.0  374.0  617.0  NaN  NaN   
6    NaN    NaN    NaN    NaN    NaN    NaN  103.0  115.0    NaN  NaN  NaN   
7    NaN    NaN    NaN    NaN    NaN    NaN  468.0  447.0    NaN  NaN  NaN   
8    NaN    NaN    NaN    NaN    NaN    NaN    NaN  114.0  100.0  NaN  NaN   
9    NaN    NaN    NaN    NaN    NaN    NaN    NaN    5.0    5.0  NaN  NaN   

                                                                         \
  2002 2003      2004      2005      2006      2007      2008      2009   
0  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2  NaN  NaN  0.135997  0.161524  0.362020  0.401793  0.266849  0.457290   
3  NaN  NaN  0.687261  1.145443  0.991362  1.241278  2.017095  1.660664   
4  NaN  NaN       NaN       NaN       NaN       NaN       NaN  0.331329   
5  NaN  NaN       NaN       NaN       NaN       NaN       NaN  2.357967   
6  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   
7  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9  NaN  NaN       NaN       NaN       NaN       NaN       NaN       NaN   

                                                                         \
       2010      2011      2012      2013      2014      2015      2016   
0       NaN       NaN  0.284704  0.209345  0.449798  0.366151  0.379113   
1       NaN       NaN  1.026532  0.877596  2.487979  2.3

### Exporter les données nettoyées

In [5]:
# Save cleaned dataset
df.to_csv("./datasets/clean_data.csv", index=None, header=True)

### Visualize Data

In [6]:
df.describe()

Count                                                          \
               2000          2001          2002          2003          2004   
count    118.000000    126.000000    128.000000    132.000000    158.000000   
mean    1853.144068   1756.944444   1775.570312   1712.303030   1387.379747   
std     6310.328587   6313.509142   6386.937989   6137.534047   5398.421033   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       26.000000     21.000000     26.500000     23.750000     12.000000   
50%       81.500000     69.500000     65.000000     70.000000     41.500000   
75%      326.250000    275.250000    320.500000    319.250000    230.000000   
max    41600.000000  44052.000000  45787.000000  47102.000000  44535.000000   

                                                                             \
               2005          2006          2007          2008          2009   
count    168.000000    192.000000    202.000000    228.000000    234.000000   
mean    1332.369048   1266.437500   1201.668317   1154.232456   1113.081197   
std     5047.603825   4809.226848   4564.440058   4371.428379   4389.398790   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       17.750000     11.750000     13.500000     13.000000     11.250000   
50%       61.000000     45.500000     54.500000     61.000000     49.000000   
75%      325.000000    282.250000    290.250000    352.500000    269.500000   
max    43678.000000  45080.000000  43906.000000  46042.000000  47131.000000   

                                                                            \
               2010          2011          2012          2013         2014   
count    240.000000    236.000000    234.000000    228.000000    226.00000   
mean    1138.883333   1177.025424   1133.042735   1064.539474   1073.40708   
std     4446.473799   4538.407856   4662.573303   4566.957230   4737.25501   
min        0.000000      0.000000      0.000000      0.000000      0.00000   
25%       15.000000     14.750000     15.250000     15.000000     13.25000   
50%       61.000000     59.500000     52.500000     57.500000     54.50000   
75%      298.250000    323.750000    291.000000    254.500000    222.50000   
max    47777.000000  47650.000000  51591.000000  51973.000000  54786.00000   

                                                                             \
               2015          2016          2017          2018          2019   
count    231.000000    226.000000    238.000000    233.000000    240.000000   
mean    1077.593074   1257.123894   1208.189076   1158.892704   1067.920833   
std     4644.600908   4972.746633   5015.465108   4828.502784   4341.440031   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       15.000000     17.000000     18.000000     16.000000     15.750000   
50%       62.000000     68.000000     58.500000     63.000000     58.500000   
75%      289.000000    444.750000    389.750000    366.000000    263.750000   
max    53467.000000  56465.000000  58750.000000  51334.000000  40268.000000   

                                         Rate                          \
               2020          2021        2000        2001        2002   
count    230.000000    194.000000  118.000000  126.000000  128.000000   
mean    1036.969565   1064.654639    7.351927    7.093214    8.424719   
std     4511.418285   4122.984087   15.203305   15.061349   16.740331   
min        0.000000      0.000000    0.000000    0.000000    0.000000   
25%       14.250000     12.250000    1.232116    1.194306    1.274867   
50%       62.500000     55.000000    2.415010    2.435052    2.511939   
75%      259.000000    226.750000    5.303030    5.019328    8.211618   
max    43780.000000  31263.000000  126.425681  129.621224  130.291396   

                                                                               \
             2003        2004        2005       

## Quels sont les pays les plus meurtriers ?

### Top 10 des pays avec le plus grand nombre de victimes tout sexe confondus

In [7]:
# Victim counts by sex 
group_df = df.groupby(by=[('Info', 'Sex')]).sum()
group_df[['Count']]

# Sort the dataframe by the highest count of victims in descending order
group_df = df.groupby(by=[('Info', 'Country')]).sum()
group_df
df_sorted = group_df.sort_values(by=[('Count', 'Total')], ascending=False)
df_sorted[['Count']].head(10)

Count                             \
                                       2000     2001     2002     2003   
(Info, Country)                                                          
Brazil                              45343.0  47903.0  49657.0  51040.0   
India                               48167.0  46420.0  45736.0  44331.0   
Russian Federation                  41090.0  42921.0  44252.0  41764.0   
Mexico                              10737.0  10285.0  10088.0  10087.0   
Colombia                            26539.0  27840.0  28387.0  23523.0   
United States of America            15586.0  19033.0  16229.0  16528.0   
Venezuela (Bolivarian Republic of)   8022.0   7960.0   9617.0  11342.0   
South Africa                            0.0      0.0      0.0      0.0   
Iraq (Central Iraq)                     0.0      0.0      0.0      0.0   
Guatemala                               0.0      0.0      0.0      0.0   

                                                                        \
                                       2004     2005     2006     2007   
(Info, Country)                                                          
Brazil                              48369.0  47564.0  49102.0  47682.0   
India                               46490.0  45095.0  44961.0  45362.0   
Russian Federation                  39256.0  35636.0  28844.0  25377.0   
Mexico                               9329.0   9921.0  10452.0   8867.0   
Colombia                            20210.0  18111.0  17479.0  17198.0   
United States of America            16148.0  16740.0  17309.0  17128.0   
Venezuela (Bolivarian Republic of)   9719.0   9964.0  12257.0  13156.0   
South Africa                            0.0      0.0  19106.0  18400.0   
Iraq (Central Iraq)                     0.0      0.0      0.0      0.0   
Guatemala                            4507.0   5338.0   5885.0   5781.0   

                                                                        \
                                       2008     2009     2010     2011   
(Info, Country)                                                          
Brazil                              50069.0  51396.0  52243.0  52166.0   
India                               45999.0  45824.0  46460.0  47640.0   
Russian Federation                  18188.0  17659.0  16690.0  16175.0   
Mexico                              14006.0  19803.0  25703.0  26950.0   
Colombia                            16140.0  15817.0  15459.0  16125.0   
United States of America            16465.0  15399.0  14722.0  14661.0   
Venezuela (Bolivarian Republic of)  14589.0  13985.0  13080.0  14098.0   
South Africa                        18084.0  16767.0  15893.0  15554.0   
Iraq (Central Iraq)                  4452.0   2458.0   2553.0   2518.0   
Guatemala                            6292.0   6498.0   5960.0   5681.0   

                                                                        \
                                       2012     2013     2014     2015   
(Info, Country)                                                          
Brazil                              56314.0  56740.0  59620.0  58086.0   
India                               47478.0  45878.0  47356.0  44373.0   
Russian Federation                  16247.0  16033.0  16516.0  17811.0   
Mexico                              25750.0  22928.0  19911.0  20676.0   
Colombia                            16433.0  16359.0  17990.0  17314.0   
United States of America            14856.0  14319.0  14164.0  15883.0   
Venezuela (Bolivarian Republic of)  16072.0      0.0      0.0  15337.0   
South Africa                            0.0      0.0      0.0      0.0   
Iraq (Central Iraq)                  2594.0   3228.0   3029.0      0.0   
Guatemala                            5155.0   6074.0   5549.0   5470.0   

                                                                        \
                                       2016     2017     2018     2019   
(Info, Country)                       

### Top 10 des pays les plus féminicides (Nombre de victimes)

In [8]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Female']
filtered_df.head(10)

# Sort the dataframe by the highest count of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Count', 'Total')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Count', 'Total')]]
df_sorted.head(10)

,Info,Count
,Country,Total
173,India,368897.0
229,Russian Federation,127340.0
60,Brazil,88784.0
140,United States of America,80760.0
102,Mexico,50573.0
68,Colombia,31415.0
36,South Africa,26725.0
90,Guatemala,13649.0
132,Venezuela (Bolivarian Republic of),11392.0


### Top 10 des pays les plus homicides (Nombre de victimes masculines)

In [9]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Male']
filtered_df.head(10)

# Sort the dataframe by the highest count of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Count', 'Total')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Count', 'Total')]]
df_sorted.head(10)

,Info,Count
,Country,Total
61,Brazil,1006754.0
174,India,615455.0
103,Mexico,405856.0
230,Russian Federation,385161.0
69,Colombia,357596.0
141,United States of America,286672.0
133,Venezuela (Bolivarian Republic of),189342.0
37,South Africa,158797.0
194,Iraq (Central Iraq),102075.0


### Top 10 des pays les plus féminicides (Taux de meurtre moyen national)

In [10]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Female']
filtered_df.head(10)

# Sort the dataframe by the highest rate of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Rate', 'Mean')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Rate', 'Mean')]]
df_sorted.head(10)

,Info,Rate
,Country,Mean
82,El Salvador,22.672727
98,Jamaica,18.748425
36,South Africa,17.620295
96,Honduras,16.968470
90,Guatemala,15.311958
120,Saint Vincent and the Grenadines,14.947932
229,Russian Federation,14.288587
56,Belize,13.805505
92,Guyana,12.733079


### Top 10 des pays les plus féminicides (Taux de meurtre national)

In [11]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Female']
filtered_df.head(10)

# Sort the dataframe by the highest rate of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Rate', 'Total')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Rate', 'Total')]]
df_sorted.head(10)

,Info,Rate
,Country,Total
82,El Salvador,204.054544
98,Jamaica,168.735823
229,Russian Federation,164.318750
96,Honduras,144.231991
90,Guatemala,137.807625
68,Colombia,137.294728
120,Saint Vincent and the Grenadines,112.109490
92,Guyana,108.231167
52,Bahamas,104.664286


### Top 10 des pays les plus homicides (Taux de meurtre moyen masculin national)

In [12]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Male']
filtered_df.head(10)

# Sort the dataframe by the highest rate of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Rate', 'Mean')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Rate', 'Mean')]]
df_sorted.head(10)

,Info,Rate
,Country,Mean
83,El Salvador,202.553722
97,Honduras,188.364661
99,Jamaica,171.154700
133,Venezuela (Bolivarian Republic of),159.444915
69,Colombia,138.845282
129,United States Virgin Islands,138.050756
117,Saint Kitts and Nevis,126.699768
57,Belize,114.167667
37,South Africa,111.985498


### Top 10 des pays les plus homicides (Taux de meurtre masculin national)

In [13]:
# Victim counts by sex 
filtered_df = df[df[('Info', 'Sex')] == 'Male']
filtered_df.head(10)

# Sort the dataframe by the highest rate of female victims in descending order
df_sorted = filtered_df.sort_values(by=[('Rate', 'Total')], ascending=False)
df_sorted = df_sorted[[('Info', 'Country'), ('Rate', 'Total')]]
df_sorted.head(10)

,Info,Rate
,Country,Total
83,El Salvador,1822.983499
97,Honduras,1601.099621
69,Colombia,1596.720745
99,Jamaica,1540.392299
133,Venezuela (Bolivarian Republic of),1355.281776
61,Brazil,1043.046477
91,Guatemala,996.863286
129,United States Virgin Islands,966.355291
53,Bahamas,955.396919


#### Courbe évolution du nombre de victime par année

In [19]:
filtered_df = df[df[('Info', 'Country')] == 'Mauritius']

group_df = filtered_df.groupby(by=[('Info', 'Country')]).sum()

group_df = group_df[['Info', 'Count']]
group_df


Info                                        \
                       Region                             Subregion   
(Info, Country)                                                       
Mauritius        AfricaAfrica  Sub-Saharan AfricaSub-Saharan Africa   

                                                          Count            \
                        Sex                        Source  2000 2001 2002   
(Info, Country)                                                             
Mauritius        FemaleMale  MD/CTS/NSO/SDGMD/CTS/NSO/SDG  26.0  0.0  0.0   

                                                                             \
                 2003  2004  2005  2006  2007  2008  2009  2010  2011  2012   
(Info, Country)                                                               
Mauritius        39.0  35.0  37.0  50.0  45.0  48.0  38.0  34.0  32.0  28.0   

                                                                              \
                 2013  2014  2015  2016  2017  2018  2019  2020  2021  Total   
(Info, Country)                                                                
Mauritius        35.0  19.0  21.0  21.0  33.0  37.0  27.0  35.0  34.0  674.0   

                            
                      Mean  
(Info, Country)             
Mauritius        64.190476

In [ ]:
sns.scatterplot(data=group_df,x='Count' , y='weight' , hue='cluster')